# Module Imports

In [1]:
import os

import pandas as pd
import numpy as np

# Loading the data

##### Using OS module to determine filepaths in OS independent manner as backslashes used on Windows and forward slashes used on Unix

In [4]:
working_directory = os.path.abspath('.')

In [76]:
training_data_location = os.path.join(working_directory, 'Data', 'train.csv')
testing_data_location = os.path.join(working_directory, 'Data', 'test.csv')

In [77]:
titanic_training_data = pd.read_csv(training_data_location)
titanic_testing_data = pd.read_csv(testing_data_location)

# Initial data exploration

In [78]:
titanic_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Training data consist of the following columns:

    1) Passenger ID: Assuming this is unique
    2) Survived: This is a true(1)/false(0) of whether or not the passenger has survived
    3) Pclass: class of ticket, this is either lower (3), middle (2) or upper (1)
    4) Name: Passenger name as a string. It may be worth splitting passenger name into surnames to see if we can group by families
    5) Sex: categorical of either male or female
    6) SibSp: This is the number of siblings and spouses on-board the titanic
    7) Parch: This is the number of parents and children on-board the titanic
    8) Ticket number: I assume this is unique --> I wonder if there is any processing that can be done to extract more information.
    9) Fare: price paid for ticket
    10) Cabin: Cabin number, maybe this is something to do with location upon the titanic..may require some processing. Or maybe we can make a new column which represents whether or not a person has a Cabin.
    11) Embarked: Location at which person embarked the titanic. S is Southampton, Q is queenstown and C is Cherbourg


## Data Preparation

Based off previous analysis the Cabin column needs to be processed to obtain information about whether or not a person had a cabin and also the location of their cabin.

In [79]:
titanic_training_data['Processed_Cabin'] = temp_df.Cabin.str.get(0).fillna('NoCabin')
titanic_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Processed_Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NoCabin
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NoCabin
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NoCabin


In [80]:
titanic_testing_data['Processed_Cabin'] = temp_df.Cabin.str.get(0).fillna('NoCabin')
titanic_testing_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Processed_Cabin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,NoCabin
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,C
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,NoCabin
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,C
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,NoCabin


In [81]:
titanic_training_data['Family_Name'] = temp_df.Name.str.split(',').str.get(0)
titanic_testing_data['Family_Name'] = temp_df.Name.str.split(',').str.get(0)

Worked out family names incase I want to include that in model.

## First Model

In [82]:
y_train = titanic_training_data.Survived

In [83]:
titanic_training_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Processed_Cabin',
       'Family_Name'],
      dtype='object')

Based off our previous analysis it looks like Sex and PClass had a big impact on survivability so let's use those for now to keep things simple.

In [84]:
x_variables = ['Sex', 'Pclass']

x_test = titanic_testing_data[x_variables]
x_test = pd.get_dummies(x_test, drop_first=True) # Convert categorical to binary

x_train = titanic_training_data[x_variables]
x_train = pd.get_dummies(x_train, drop_first=True) # Convert categorical to binary
x.head()

,Pclass,Sex_male
0,3,1
1,1,0
2,3,0
3,1,0
4,3,1


In [85]:
from sklearn.tree import DecisionTreeRegressor

sex_class_model = DecisionTreeRegressor(random_state=1)
sex_class_model.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [86]:
sex_class_model.predict(x_train)

array([0.13544669, 0.96808511, 0.5       , 0.96808511, 0.13544669,
       0.13544669, 0.36885246, 0.13544669, 0.5       , 0.92105263,
       0.5       , 0.96808511, 0.13544669, 0.13544669, 0.5       ,
       0.92105263, 0.13544669, 0.15740741, 0.5       , 0.5       ,
       0.15740741, 0.15740741, 0.5       , 0.36885246, 0.5       ,
       0.5       , 0.13544669, 0.36885246, 0.5       , 0.13544669,
       0.36885246, 0.96808511, 0.5       , 0.15740741, 0.36885246,
       0.36885246, 0.13544669, 0.13544669, 0.5       , 0.5       ,
       0.5       , 0.92105263, 0.13544669, 0.92105263, 0.5       ,
       0.13544669, 0.13544669, 0.5       , 0.13544669, 0.5       ,
       0.13544669, 0.13544669, 0.96808511, 0.92105263, 0.36885246,
       0.36885246, 0.92105263, 0.13544669, 0.92105263, 0.13544669,
       0.13544669, 0.96808511, 0.36885246, 0.13544669, 0.36885246,
       0.13544669, 0.92105263, 0.13544669, 0.5       , 0.13544669,
       0.15740741, 0.5       , 0.15740741, 0.13544669, 0.13544

In [112]:
y_test = sex_class_model.predict(x_test)
type(y_test)

numpy.ndarray

In [126]:
output_df = titanic_testing_data[['PassengerId']]
output_df.loc[:, 'Survived'] = y_test.copy()
output_df.head()

,PassengerId,Survived
0,892,0.135447
1,893,0.500000
2,894,0.157407
3,895,0.135447
4,896,0.500000


In [133]:
output_df.loc[:, 'Survived'] = output_df.Survived.round(decimals = 0).astype(int).copy()

In [134]:
output_df.to_csv('predictions.csv', index=False)